## Keystone Project - Portfolio Optimization, CAPM & Black-Litterman

__Assignment / Goal:__

You want to invest in a (optimized) Portfolio consisting of the 30 Dow Jones Stocks. 

1. Get all 30 Dow Jones Constituents from the Web (e.g.  "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")

2. Get daily prices (last 3Y) and Market Cap from Yahoo Finance and prepare daily returns and the covariance matrix. Get prices/returns for the Market Portfolio (S&P 500 - "^GSPC") as well. 

3. Calculate beta (3Y monthly) and identify all cyclical (beta > 1) and defensive (beta < 1) stocks. 

4. Calculate CAPM returns for all stocks. For the risk-free Asset, use the 13 Week US Treasury Bill ("^IRX"). For the expected Market Return, analyze two different scenarios:
- Bull Market (10%)
- Bear Market (0%)

5. Use CAPM returns (see 4.) for Portfolio Optimization (maximize Sharpe Ratio). Optimization Bounds: All Stocks shall have a minimum weight of 1% and a maximum weight of 25%!

6. Calculate and interpret the weighted average portfolio beta in
- Bull Market scenario (10%)
- Bear Market scenario (0%)

7. Calculate implied stock returns with Reverse Optimization (assume 10% market return) and compare with CAPM returns.

8. Add your own views and run a Black-Litterman Optimization. 

# --------------------SOLUTION------------------------------

## Getting started

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:.2f}'.format
plt.style.use("seaborn-v0_8")

In [ ]:
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"

In [ ]:
df = pd.read_html(url)[1]
df

In [ ]:
df = df[["Company", "Symbol", "Industry"]].copy()
df

In [ ]:
df.set_index("Symbol", inplace = True)
df

In [ ]:
symbols = df.index.to_list()
symbols

In [ ]:
index = "^GSPC"

In [ ]:
symbols.append(index)
symbols

In [ ]:
prices = yf.download(symbols, period = "3y", interval = "1d")
prices

In [ ]:
returns = prices["Close"].pct_change()
returns

In [ ]:
cov_matrix = returns[df.index].cov() * 252
cov_matrix

In [ ]:
count = 0
for ticker in df.index:
    count += 1
    print(count, end = '\r')
    try:
        #info = yf.Ticker(ticker).get_info()
        #df.loc[ticker, "beta"] = info["beta"]
        fast_info = yf.Ticker(ticker).fast_info
        df.loc[ticker, "mcap"] = fast_info["market_cap"]
    except Exception as e:
        print(ticker, e)

In [ ]:
df

## Cyclical Stocks vs. Defensive Stocks

In [ ]:
returns_m = prices.Close.resample("M").last().pct_change() # monthly returns
returns_m

In [ ]:
df["beta"] = returns_m.cov()["^GSPC"] / returns_m["^GSPC"].var() # 3Y monthly beta
df

__Cyclical Stocks__

In [ ]:
df.loc[df.beta > 1].sort_values(by = "beta", ascending = False)

__Defensive Stocks__

In [ ]:
df.loc[df.beta < 1].sort_values(by = "beta", ascending = True)

## CAPM Returns and Portfolio Optimization

In [ ]:
df

__Risk-free Rate__

In [ ]:
rfr = yf.download(tickers = "^IRX").Close.dropna().iloc[-1] / 100 #13 Week US Treasury Bill
rfr

__Market Return Forecast (bullish)__

In [ ]:
rm = 0.10

__Market Return Forecast (bearish)__

In [ ]:
# rm = 0.00

In [ ]:
df["CAPM_ret"] = rfr + (rm - rfr) * df.beta
df

In [ ]:
def port_ret(weights):
    ret = df.CAPM_ret.dot(weights)
    return ret

In [ ]:
def port_vol(weights):
    risk = np.sqrt(weights.dot(cov_matrix).dot(weights))
    return risk

In [ ]:
def sharpe(weights): 
     return -(port_ret(weights)-rfr) / port_vol(weights) 

In [ ]:
import scipy.optimize as sco

In [ ]:
noa = len(df)
noa

In [ ]:
eweigths = np.full(noa, 1/noa)
eweigths

In [ ]:
#constraint: weights must sum up to 1 -> sum of weights - 1 = 0
cons = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})

In [ ]:
bnds =  tuple((0.01, 0.25) for x in range(noa))
bnds

In [ ]:
opts = sco.minimize(sharpe, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
optimal_weights = opts["x"]
optimal_weights

In [ ]:
df["CAPM_weights"] = pd.Series(data = optimal_weights, index = df.index)

In [ ]:
df.sort_values(by = "CAPM_weights", ascending = False)

In [ ]:
weighted_average_beta = df.beta.mul(df.CAPM_weights).sum()
weighted_average_beta

- __Expected Bull Market: High Beta Portfolio__
- __Expected Bear Market: Low Beta Portfolio__

## Reverse Optimization (Black-Litterman)

In [ ]:
df

In [ ]:
cov_matrix

In [ ]:
df["mcap_weights"] = df.mcap.div(df.mcap.sum())
df

In [ ]:
rm = 0.1

In [ ]:
# normalization factor
nf = (rm - rfr) / (df.mcap_weights.T.dot(cov_matrix).dot(df.mcap_weights))
nf

In [ ]:
# implied returns (Black-Litterman)
df["BL_returns"] = cov_matrix.dot(df.mcap_weights) * nf + rfr
df

In [ ]:
# CAPM Returns & BL Returns are implied returns (market expectations)
df["blended_returns"] = (df.CAPM_ret + df.BL_returns) / 2
df

In [ ]:
# investor opinions
df["returns_final"] = df["blended_returns"].copy()
df.loc["IBM", "returns_final"] = df.loc["IBM", "blended_returns"] + 0.03
df.loc["NKE", "returns_final"] = df.loc["NKE", "blended_returns"] + 0.02
df.loc["INTC", "returns_final"] = df.loc["INTC", "blended_returns"] - 0.03
df.loc["MCD", "returns_final"] = df.loc["MCD", "blended_returns"] - 0.02

In [ ]:
df

In [ ]:
eweigths = np.full(noa, 1/noa)
eweigths

In [ ]:
bnds =  tuple((0.01, 0.25) for x in range(noa))
bnds

In [ ]:
def port_ret(weights):
    ret = df.returns_final.dot(weights)
    return ret

In [ ]:
opts = sco.minimize(sharpe, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
optimal_weights = opts["x"]
optimal_weights

In [ ]:
df["final_weights"] = pd.Series(data = optimal_weights, index = df.index)

In [ ]:
df.sort_values(by = "final_weights", ascending = False)